<a href="https://colab.research.google.com/github/adeepH/ANN_from_scratch/blob/master/XNOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building an Artificial Neural Network from Scratch

To learn the outcomes of XNOR gate

In [0]:
import numpy as np
from bokeh.plotting import figure,output_notebook,show
output_notebook()

In [0]:
def tanh(x,derivative=False):
  if derivative:
    1-x**2
  return np.tanh(x)

In [0]:
def sigmoid(x,derivative=False):
  if derivative:
    return x*(1-x)
  return 1/(1+np.exp(-x))

In [0]:
#Loss function 
#MSE : Mean Square Error
def mse(y_pred,y,derivative=False):
  if derivative:
    return y_pred*(y_pred-y)
  return 0.5*(y_pred-y)**2

In [0]:
class Layer:
  def __init__(self, weights_shape, previous_result=None):
    self.previous_result = previous_result
    self.weights = self.init_weights(weights_shape)
    self.out = None
    self.bias = np.random.uniform(-1,1) #if the XNOR is biased 
  
  def init_weights(self,weights_shape):
    return np.random.uniform(-1,1,weights_shape)
  
  def compute_output(self,previous_result,activation_function):
    self.previous_result = previous_result
    self.out = activation_function(self.previous_result.dot(self.weights)+self.bias)

In [0]:
class ANN:
    def __init__(self, input_values, output_values, learning_rate=1e-1, activation_function=sigmoid):
        self.input_values = input_values
        self.layers = []
        self.output_values = output_values
        self.lr = learning_rate
        self.activation_function = activation_function
            
    def add_layer(self, nb_nodes):
        if len(self.layers)==0:
            weights_shape = (self.input_values.shape[1], nb_nodes)
            previous_result = self.input_values
            layer = Layer(weights_shape, previous_result)
        else:
            weights_shape = (self.layers[-1].weights.shape[1], nb_nodes)
            layer = Layer(weights_shape, self.layers[-1].out)
        self.layers.append(layer)
        
    def feed_forward(self):
        previous_result = self.layers[0].previous_result
        for layer in self.layers:
            layer.compute_output(previous_result, self.activation_function)
            previous_result = layer.out
    
    def back_prop(self):
        previous_layer = last_layer = self.layers[-1]
        d = mse(last_layer.out, self.output_values, True) * self.activation_function(last_layer.out, True)
        last_layer.weights -= self.lr * last_layer.previous_result.T.dot(d)
        last_layer.bias -= self.lr * np.mean(d)
        for layer in np.flip(self.layers, axis=0)[1:]:
            d = d.dot(previous_layer.weights.T) * self.activation_function(layer.out, True)
            layer.weights -= self.lr * layer.previous_result.T.dot(d)
            layer.bias -= self.lr * np.mean(d)
            previous_layer = layer
        
    def train(self, nb_epochs=65000):
        self.layers = np.array(self.layers)
        error_list = []
        for i in range(nb_epochs):
            self.feed_forward()
            self.back_prop()
            error_list.append(np.mean(mse(self.layers[-1].out, self.output_values)))
        return error_list

    def predict(self, input_values):
        previous_result = np.array(input_values)
        for layer in self.layers:
            layer.compute_output(previous_result, self.activation_function)
            previous_result = layer.out
        return previous_result

In [0]:
train = np.array([
    [0,0],
    [0,1],
    [1,0],
    [1,1]
])

train_result = np.array([1,0,0,1]).reshape((-1,1))


In [0]:
#Executing the class ANN
ann = ANN(train,train_result,learning_rate=1e-1,activation_function=sigmoid)
ann.add_layer(3)
ann.add_layer(1)
error = ann.train(nb_epochs = 70000)

In [11]:
np.around(ann.predict(train))

array([[1.],
       [0.],
       [0.],
       [1.]])

In [15]:
p = figure(title= 'Error evaluation during learning ')
p.line(np.arange(len(error)),error,line_color='firebrick',legend='Error')
show(p)